# Importing the libraries

In [304]:
import requests  # library to make HTTP requests
from bs4 import BeautifulSoup  # library for web scraping
from googlesearch import search  # library for performing Google searches
import concurrent.futures  # library for parallelism
import threading  # library for thread synchronization
import pickle  # library for saving data to a file
import re # library for regular expressions
import nltk # library for natural language processing
import time # library for time
import json # library for JSON
import math # library for mathematical operations
from nltk.corpus import stopwords # library for stop words
from nltk.stem import PorterStemmer # library for stemming
from nltk.tokenize import word_tokenize # library for tokenization

nltk.download('stopwords') # download stop words
nltk.download('punkt') # download tokenizer

stop_words = set(stopwords.words('english')) # set of stop words
ps = PorterStemmer() # stemmer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ishikaj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishikaj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Define the search queries

In [305]:
query1 = "Forests of India"
query2 = "Tiger Density in India"
query3 = "Night safari in Forests"
query4 = "Tiger AND Safari AND Leopard"

# Function to Crawl the top 20 web pages from Google search engine
This function takes a query as input and returns a list of URLs obtained from Google search results. The function uses the `search()` function from the `googlesearch` library to perform the search and retrieve the URLs. The search results are limited to 20 and the language is set to "en" (English). The URLs are stored in the `url_list` variable and returned by the function.

In [306]:
#  crawl the top 20 web pages from google search engine
def crawl(query):
    """
    Returns a list of urls for the given query

    Parameters:
    query (str): The query to be searched on google

    Returns:
    url_list (list): A list of urls for the given query

    """
    
    #  get the list of urls
    url_list = []

    #  loop through the urls and get the text
    for url in search(query, num_results=50, lang="en"):
        url_list.append(url)

    print("length of url list: ", len(url_list))
    #  return the list of urls
    return url_list

### Function to Get the first 3 paragraphs from the URL

This function takes a URL as input and returns the first three non-empty paragraphs from the page as a list of strings. The function uses the `requests` library to retrieve the HTML code of the page and the `BeautifulSoup` library to parse the HTML and the first three non-empty `<p>` tags are retrieved and returned as a list of strings.

In [307]:
# get the first 3 paragraph from the url
def get_text_paragraphs(url):
    """
    Returns the first 3 paragraphs from the url

    Parameters:
    url (str): url of the web page

    Returns:
    list: list of first 3 paragraphs

    """


    #  get the first 3 paragraphs from the url
    paragraphs = []
    try:
        #  get the html code from the url, if url taking more than 10 seconds to respond, then skip the url
        page = requests.get(url, timeout=10)

        if page.status_code != 200:
            print("Error in getting text")
            return []
       
        #  parse the html code using beautiful soup library
        soup = BeautifulSoup(page.content, 'html.parser')

        #  get the body tag
        body = soup.find('body')

        #  if body tag is not present, then skip the url
        if body is None:
            print("Body tag not found")
            return []
        

        #  get first 3 non empty <p> tags 
        p_tags = body.find_all('p')

        # if there are nested <p> tags, then get the text from the inner most <p> tag
        for i in range(len(p_tags)):
            while p_tags[i].find('p') is not None:
                p_tags[i] = p_tags[i].find('p')

        #  if there are less than 3 <p> tags, then get all the <span> tags
        if len(p_tags) < 3:
            p_tags = body.find_all('span')

            # if there are nested <span> tags, then get the text from the inner most <span> tag
            for i in range(len(p_tags)):
                while p_tags[i].find('span') is not None:
                    p_tags[i] = p_tags[i].find('span')
        
        #  if there are less than 3 <span> tags, then get all the <div> tags
        if len(p_tags) < 3:
            p_tags = body.find_all('div')

            # if there are nested <div> tags, then get the text from the inner most <div> tag
            for i in range(len(p_tags)):
                while p_tags[i].find('div') is not None:
                    p_tags[i] = p_tags[i].find('div')
            
        
        # remove empty tags
        p_tags = [p for p in p_tags if p.text.strip() != '']

        # remove paragraphs with non english and non number characters
        p_tags = [p for p in p_tags if re.search('[a-zA-Z0-9]', p.text.strip()) is not None]

        #  get first 3 tags
        p_tags = p_tags[:3]

        #  get the text from the tags
        paragraphs = [p.text for p in p_tags]

    except:
        print("Here : Error in getting text")
        return []
    

    return paragraphs

## Function to extract the first three paragraphs from each of the webpages.

This function takes a search query as input and returns a list of the first three paragraphs from each of the top 20 websites returned by a Google search for the query. The function first calls the `crawl()` function to get a list of URLs for the query. The function then loops through the URLs and calls the `get_text()` function to retrieve the first three paragraphs from each website. The paragraphs from all the websites are combined into a single list and returned by the function.d.

In [308]:
def get_paragraphs(query):

    """
    Returns a dictionary of urls and their first 3 paragraphs

    Parameters:
    query (str): The query to be searched on google

    Returns:
    pages (dict): A dictionary of urls and their first 3 paragraphs

    """


    #  get the list of urls
    print(f"Query: {query}")

    url_list = crawl(query)

    # remove duplicate urls
    url_list = list(dict.fromkeys(url_list))

    #  get the first 3 paragraphs from each url in parallel
    pages = {}
    print(f"Retrieving pages...")

    # define a lock to prevent race conditions when updating pages dict
    lock = threading.Lock()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # submit a separate thread to retrieve the first 3 pages for each URL
        future_to_url = {executor.submit(get_text_paragraphs, url): url for url in url_list}

        for future in concurrent.futures.as_completed(future_to_url):
            #  if pages dic has 20 entries, then stop
            if len(pages) == 20:
                break

            url = future_to_url[future]

            try:
                para = future.result()
                if len(para) == 3:
                    print(f"Retrieved {url} is ok")
                    # acquire lock before updating pages dict
                    lock.acquire()

                    pages[url] = para

                    # release lock after updating pages dict
                    lock.release()
                else:
                    print(f"Error in getting text from {url}")
            except Exception as exc:
                print(f"Exception occurred while getting text from {url}: {exc}")

    print(f"Retrieved {len(pages)} pages")
    return pages


## Function to clean the text

This function takes a list of strings as input and returns a list of strings after removing the punctuations, numbers, and special characters from the strings. The function uses the `re` library to remove the punctuations, numbers, and special characters from the strings.

In [309]:
# function to clean the text
def clean_text(text):

    """
    Returns a list of words after removing stop words and special characters

    Parameters:
    text (str): The text to be cleaned

    Returns:
    list: A list of words after removing stop words and special characters

    """

    # remove all the special characters
    text = re.sub(r'\W', ' ', text)

    # remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # remove numbers
    text = re.sub(r'\d+', '', text)

    # covert to lower case
    text = text.lower()

    # remove all the stopwords
    text = [word for word in word_tokenize(text) if word not in stop_words]

    #  join the words
    text = ' '.join(text)

    return text



# Function to create inverted index

This function takes a list of documents and name of document as input and returns an inverted index for the documents. The function first creates a dictionary `inverted_index` to store the inverted index. The function loops through the words in the documents and adds the document name to the list of documents for the word in the inverted index. The function returns the inverted index.

In [310]:
# Write a program in Python to build an inverted index of these document
def create_inverted_index(documents, doc_names):

    """
    Returns an inverted index of the documents

    Parameters:
    documents (list): A list of documents
    doc_names (list): A list of document names

    Returns:
    dict: An inverted index of the documents

    """
    
    #  get the list of words in the documents
    words = []
    for doc in documents:
        words.extend(doc.split())

    #  remove duplicate words
    words = list(dict.fromkeys(words))

    #  create an inverted index
    inverted_index = {}
    for word in words:
        inverted_index[word] = []
        for i in range(len(documents)):
            if word in documents[i].split():
                inverted_index[word].append(doc_names[i])

    return inverted_index

Find all the paragraphs from the top 20 websites for the query1.

In [124]:
pages1 = get_paragraphs(query1)

Query: Forests of India
length of url list:  52
Retrieving pages...
Retrieved http://edugreen.teri.res.in/explore/forestry/history.htm is ok
Retrieved https://www.vedantu.com/question-answer/what-are-the-different-types-of-forests-in-india-5ecf59425c02fb0719ca96f1 is ok
Error in getting text
Error in getting text from https://www.toppr.com/ask/question/describe-the-forests-of-india/
Retrieved https://fsi.nic.in/scheme-of-classification is ok
Error in getting text
Error in getting text from https://india.mongabay.com/2022/01/the-state-of-indias-forests-losing-forests-gaining-plantations/
Retrieved https://www.india.gov.in/topics/environment-forest is ok
Retrieved https://www.drishtiias.com/to-the-points/paper1/types-of-forests-in-india is ok
Retrieved https://pib.gov.in/PressReleasePage.aspx?PRID=1789635 is ok
Retrieved https://www.clubmahindra.com/blog/experience/top-6-forests-in-india is ok
Here : Error in getting text
Error in getting text from https://ifs.nic.in/
Error in getting te

FInd all the paragraphs from the top 20 websites for the query2.

In [126]:
pages2 = get_paragraphs(query2)

Query: Tiger Density in India
length of url list:  52
Retrieving pages...
Retrieved https://byjus.com/ias-questions/which-national-park-has-highest-number-of-tigers/ is ok
Error in getting text
Error in getting text from https://www.insightsonindia.com/2022/03/11/tiger-density-in-india/
Retrieved https://www.thehindu.com/news/national/other-states/Kaziranga-has-the-worlds-highest-tiger-density-report/article16373525.ece is ok
Error in getting text
Error in getting text from https://www.newindianexpress.com/nation/2020/jul/28/corbett-reserve-has-highest-tiger-density-in-india-report-2175962.html
Retrieved https://en.wikipedia.org/wiki/Tiger_reserves_of_India is ok
Here : Error in getting text
Error in getting text from https://optimizeias.com/tiger-density-in-india/
Retrieved https://www.acubeias.com/current-affairs-for-upsc/tiger-density-in-india is ok
Retrieved https://www.drishtiias.com/daily-updates/daily-news-analysis/corbett-tiger-reserve-uttarakhand-1 is ok
Retrieved https://www.

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Body tag not found


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Body tag not found


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Body tag not found
Retrieved 20 pages


Find all the paragraphs from the top 20 websites for the query3.

In [128]:
pages3 = get_paragraphs(query3)

Query: Night safari in Forests
length of url list:  51
Retrieving pages...
Retrieved https://www.deccanherald.com/national/south/environmentalists-protest-jungle-night-safari-in-wayanad-forest-1154114.html is ok
Retrieved https://www.tadobanationalparkonline.in/night-safari-in-tadoba.php is ok
Retrieved https://www.mptourism.com/night-safari-at-these-national-parks-in-madhya-pradesh.html is ok
Retrieved https://forest.mponline.gov.in/ is ok
Retrieved http://www.walkthroughindia.com/wildlife/top-5-places-proposed-night-safari-india/ is ok
Retrieved https://www.dailypioneer.com/2022/india/india---s-first-night-safari-to-come-up-in-lucknow.html is ok
Retrieved https://thelivenagpur.com/2022/10/13/night-safaris-banned-in-maha-and-mp/ is ok
Retrieved https://www.jimcorbettnationalpark.co.in/online-corbett-night-stay-booking.html is ok
Retrieved http://www.clarissaresorts.com/night-safari.php is ok
Retrieved https://timesofindia.indiatimes.com/travel/travel-news/mp-you-will-now-be-able-to-en

Find all the paragraphs from the top 20 websites for the query4.

In [129]:
pages4 = get_paragraphs(query4)

Query: Tiger AND Safari AND Leopard
length of url list:  50
Retrieving pages...
Retrieved https://curlytales.com/wildlife-safaris-in-rajasthan-that-ensures-tiger-leopard-spotting/ is ok
Retrieved https://www.tigersafariindia.co.uk/leopard-safari-and-tiger-safari-in-india/ is ok
Retrieved https://m.timesofindia.com/home/sunday-times/aside-tigers-let-the-leopard-safari-begin/articleshow/97615427.cms is ok
Retrieved https://www.responsibletravel.com/holidays/big-cat-safaris/travel-guide/siberian-tiger-and-amur-leopard-safaris is ok
Error in getting text
Error in getting text from https://www.naturesafariindia.com/tiger-safari-tours/tiger-leopards-and-lions-photographic-safari/
Error in getting text
Error in getting text from https://www.tigersafariindia.com/tour-packages/tiger-taj-leopard-safari/
Error in getting text
Error in getting text from https://www.kanha.net/tiger-safari-tours/taj-tiger-and-leopard-safari-tour-in-india/
Retrieved https://www.ranthamborenationalpark.com/tiger-leopa

#### Save the pages dictionary in a pickle file

In [132]:
# #  save paragraphs in pkl file
# with open('pages1.pkl', 'wb') as f:
#     pickle.dump(pages1, f)

# with open('pages2.pkl', 'wb') as f:
#     pickle.dump(pages2, f)

# with open('pages3.pkl', 'wb') as f:
#     pickle.dump(pages3, f)

# with open('pages4.pkl', 'wb') as f:
#     pickle.dump(pages4, f)

#### Open the pages dictionary from the pickle file

In [311]:
#  open the pkl file
with open('pages1.pkl', 'rb') as f:
    pages1 = pickle.load(f)

with open('pages2.pkl', 'rb') as f:
    pages2 = pickle.load(f)

with open('pages3.pkl', 'rb') as f:
    pages3 = pickle.load(f)

with open('pages4.pkl', 'rb') as f:
    pages4 = pickle.load(f)

### clean the text in the pages dictionary

In [312]:
#  clean the text
cleaned_pages1 = {}
for url, paragraphs in pages1.items():
    cleaned_pages1[url] = [clean_text(paragraph) for paragraph in paragraphs]

cleaned_pages2 = {}
for url, paragraphs in pages2.items():
    cleaned_pages2[url] = [clean_text(paragraph) for paragraph in paragraphs]

cleaned_pages3 = {}
for url, paragraphs in pages3.items():
    cleaned_pages3[url] = [clean_text(paragraph) for paragraph in paragraphs]

cleaned_pages4 = {}
for url, paragraphs in pages4.items():
    cleaned_pages4[url] = [clean_text(paragraph) for paragraph in paragraphs]


### create the document to save the paragraphs information

In [313]:
#  create documents for query 1 using the cleaned pages
for i in range(1, 21):
    globals()[f"d{i}_q1"] = globals()[f"cleaned_pages1"][list(globals()[f"cleaned_pages1"])[i-1]]
    globals()[f"d{i}_q1"] = ' '.join(globals()[f"d{i}_q1"])

#  create documents for query 2 using the cleaned pages
for i in range(1, 21):
    globals()[f"d{i}_q2"] = globals()[f"cleaned_pages2"][list(globals()[f"cleaned_pages2"])[i-1]]
    globals()[f"d{i}_q2"] = ' '.join(globals()[f"d{i}_q2"])

#  create documents for query 3 using the cleaned pages
for i in range(1, 21):
    globals()[f"d{i}_q3"] = globals()[f"cleaned_pages3"][list(globals()[f"cleaned_pages3"])[i-1]]
    globals()[f"d{i}_q3"] = ' '.join(globals()[f"d{i}_q3"])

#  create documents for query 4 using the cleaned pages
for i in range(1, 21):
    globals()[f"d{i}_q4"] = globals()[f"cleaned_pages4"][list(globals()[f"cleaned_pages4"])[i-1]]
    globals()[f"d{i}_q4"] = ' '.join(globals()[f"d{i}_q4"])


In [314]:
# save the documents in text files in documents folder
for i in range(1, 21):
    with open(f"documents/d{i}_q1.txt", "w") as f:
        f.write(globals()[f"d{i}_q1"])
    
    with open(f"documents/d{i}_q2.txt", "w") as f:
        f.write(globals()[f"d{i}_q2"])

    with open(f"documents/d{i}_q3.txt", "w") as f:
        f.write(globals()[f"d{i}_q3"])

    with open(f"documents/d{i}_q4.txt", "w") as f:
        f.write(globals()[f"d{i}_q4"])

In [315]:
# create a list of all the documents
documents = []
for i in range(1, 21):
    documents.append(globals()[f"d{i}_q1"])
    documents.append(globals()[f"d{i}_q2"])
    documents.append(globals()[f"d{i}_q3"])
    documents.append(globals()[f"d{i}_q4"])

#  create a list of all the document names
doc_names = []
for i in range(1, 21):
    doc_names.append(f"d{i}_q1")
    doc_names.append(f"d{i}_q2")
    doc_names.append(f"d{i}_q3")
    doc_names.append(f"d{i}_q4")

Create the inverted index for the paragraphs in the pages dictionary.

In [316]:
# create an inverted index
inverted_index = create_inverted_index(documents, doc_names)


Save the inverted index in a json file.

In [141]:

# #  save the inverted index in a text file
# with open('inverted_index.json', 'w') as file:
#     file.write(json.dumps(inverted_index))


open the inverted index from the json file.

In [81]:
#  read the inverted index from the text file
with open('inverted_index.json', 'r') as file:
    inverted_index = json.loads(file.read())

change the document names to integers for easier processing.

In [317]:
#  change the document names to integers
for word, doc_names in inverted_index.items():
    for i in range(len(doc_names)):
        doc_names[i] = int(doc_names[i][1:].split("_")[0]) + (int(doc_names[i].split("_")[1][1:]) - 1) * 20


save modified inverted index in a text file

In [318]:
import json
with open('inverted_index_modified.json', 'w') as file:
    file.write(json.dumps(inverted_index))

Sort the inverted index by the number of documents in the list.

In [319]:
# sort the inverted index by the number of documents each word appears in
sorted_inverted_index = {k: v for k, v in sorted(inverted_index.items(), key=lambda item: len(item[1]), reverse=True)}


#### Print top 3 words with the highest number of documents and bottom 3 words with the lowest number of documents

In [320]:
#  print top 3 and bottom 3 words in the inverted index
print("Top 3 words in the inverted index:")
for i in range(3):
    print(list(sorted_inverted_index.keys())[i])

print("______________________________________________________")
print()
print("Bottom 3 words in the inverted index:")
for i in range(3):
    print(list(sorted_inverted_index.keys())[-i-1])


Top 3 words in the inverted index:
india
tiger
safari
______________________________________________________

Bottom 3 words in the inverted index:
share
pack
balloon


# Merge posting-lists

Implement the merge algorithm for intersecting the postings of two terms, as well as
code to use it to process Boolean queries. When there are multiple query terms, make
sure that your algorithm uses the optimization of performing the most restrictive
intersection first.
Using your algorithm and the index you built on the web documents, process the
following queries:
1. Tiger AND Safari
2.  Wildlife AND Poaching
3.  Leopard AND Night

In [321]:
sample_queries = ['Tiger AND Safari', 'Wildlife AND Poaching', 'Leopard AND Night']

## Function to merge the posting lists

In [322]:
def merge_algo(queries):
  
  """ 
  Returns the postings list of the query using merge algorithm

    Parameters:
    queries (list): list of words in the query

    Returns:
    list: postings list of the query

  """

  answer_postings = []

  # loop through the queries and append the postings list of each query to answer_postings 
  for i in queries:
    # check if the word is in the inverted index
    if sorted_inverted_index.get(i) == None:
      print(i, " Not Present")
      # return empty list if the word is not in the inverted index
      return []
    
    else:
      
      answer_postings.append(sorted_inverted_index[i])
      
  # help restrictive intersection first
  answer_postings.sort(key=len)
  
  top = answer_postings[0]

  for i in range(1, len(answer_postings)):
    #intersection with first list
    print("happen")

    m = len(top)
    
    n = len(answer_postings[i])

    print(top)

    print(answer_postings[i])

    k = 0
    l = 0

    merged = []

    #  merge the two lists
    while(k < m and l < n):

      # if the two elements are equal, append it to the merged list
      if(top[k] == answer_postings[i][l]):

        merged.append(top[k])

        k = k + 1
        l = l + 1
      else:
        # if the element in the first list is smaller, increment the index of the first list
        if(top[k] < answer_postings[i][l]):
          k = k + 1
        # if the element in the second list is smaller, increment the index of the second list
        else:
          l = l + 1

    top = merged

  return top

In [323]:
def extract_queries(query):

    #  split the query into words
    query = query.split("AND")

    #  remove the spaces from the words
    query =  [i.strip() for i in query]

    #  clean the words
    query = [clean_text(i) for i in query]

    
    return query

## Function to process the boolean queries and return the documents

In [324]:
def process_query_1b(user_query):
    """ 
    Function to process the query and return the relevant documents

        Parameters:
        user_query (str): query entered by the user

        Returns:
        list: list of relevant documents

    """
  
    #  extract the queries from the user query
    user_query = extract_queries(user_query)

    # remove stop words from the query
    print("query" , user_query)

    retrieved_docs = merge_algo(user_query)
    # print("retrieved docs", retrieved_docs)
    
    return retrieved_docs

In [325]:
results = []
# process the queries
for query in sample_queries:
    results.append(process_query_1b(query))
print("Results:")
print(results)

query ['tiger', 'safari']
happen
[41, 62, 43, 44, 25, 45, 65, 46, 66, 47, 67, 48, 68, 49, 69, 30, 70, 51, 52, 74, 56, 57, 77, 78, 39, 59, 80]
[21, 62, 23, 44, 25, 65, 66, 27, 47, 67, 28, 48, 68, 69, 30, 51, 71, 32, 52, 13, 33, 54, 74, 35, 37, 78, 39, 40, 80]
query ['wildlife', 'poaching']
poaching  Not Present
query ['leopard', 'night']
happen
[62, 65, 66, 67, 69, 70, 71, 77, 78, 39, 80]
[41, 42, 43, 45, 46, 47, 48, 71, 32, 52, 55, 56, 57]
Results:
[[62, 44, 25, 65, 66, 47, 67, 48, 68, 69, 30, 74, 78, 39, 80], [], [71]]


#### Running the same query 100 times and calculating the average time taken to process the query.

In [326]:
#Testing provided queries

# 100 random queries

time_for_merge = []
for i in sample_queries:
    start_time_1b = time.time()
    for j in range(100):

        print("Retrieved docs for ", i)
        process_query_1b(i)

    end_time_1b = time.time()
    time_for_merge.append(end_time_1b - start_time_1b)

print("Time taken for merge algorithm: ", time_for_merge)

Retrieved docs for  Tiger AND Safari
query ['tiger', 'safari']
happen
[41, 62, 43, 44, 25, 45, 65, 46, 66, 47, 67, 48, 68, 49, 69, 30, 70, 51, 52, 74, 56, 57, 77, 78, 39, 59, 80]
[21, 62, 23, 44, 25, 65, 66, 27, 47, 67, 28, 48, 68, 69, 30, 51, 71, 32, 52, 13, 33, 54, 74, 35, 37, 78, 39, 40, 80]
Retrieved docs for  Tiger AND Safari
query ['tiger', 'safari']
happen
[41, 62, 43, 44, 25, 45, 65, 46, 66, 47, 67, 48, 68, 49, 69, 30, 70, 51, 52, 74, 56, 57, 77, 78, 39, 59, 80]
[21, 62, 23, 44, 25, 65, 66, 27, 47, 67, 28, 48, 68, 69, 30, 51, 71, 32, 52, 13, 33, 54, 74, 35, 37, 78, 39, 40, 80]
Retrieved docs for  Tiger AND Safari
query ['tiger', 'safari']
happen
[41, 62, 43, 44, 25, 45, 65, 46, 66, 47, 67, 48, 68, 49, 69, 30, 70, 51, 52, 74, 56, 57, 77, 78, 39, 59, 80]
[21, 62, 23, 44, 25, 65, 66, 27, 47, 67, 28, 48, 68, 69, 30, 51, 71, 32, 52, 13, 33, 54, 74, 35, 37, 78, 39, 40, 80]
Retrieved docs for  Tiger AND Safari
query ['tiger', 'safari']
happen
[41, 62, 43, 44, 25, 45, 65, 46, 66, 47, 6

# Adding Skip-pointers

Re-index the same set of documents crawled in part 1 with skip-pointers. For a posting
list of length P, use √P evenly-spaced skip pointers. Execute the same three queries
mentioned above 100 times each and compare the absolute total time taken to run them
for the index with skip-pointers and index without skip-pointers (created during part 1).


In [327]:
def create_skip_lists(posting_list):

    """
    Returns a skip list of the posting list

    Parameters:
    posting_list (list): A posting list

    Returns:
    skip_list (list): A skip list of the posting list

    """
    
    #  create a skip list
    skip_list = []
    
    # get the size of the posting list
    size = len(posting_list)

    # For a posting list of length P, use √P evenly-spaced skip pointers.
    # The first skip pointer is at the √Pth document, the second at the 2√Pth document, and so on.
    # The last skip pointer is at the last document in the posting list.
    skip_count = int(math.sqrt(size))

    period = size // skip_count

    curr_skip_index = period - 1

    for i in range(size):
        # if the current index is equal to the current skip index, add the document to the skip list with a skip pointer of the next skip index
        if i == curr_skip_index:

            skip_list.append([posting_list[i], curr_skip_index + period])

            #  update the current skip index
            curr_skip_index += period

        # if the current index is not equal to the current skip index, add the document to the skip list with a skip pointer of 0
        else:
            
            skip_list.append([posting_list[i], 0])

            

    return skip_list


## Function to create the inverted index with skip pointers

In [328]:
def create_skip_pointers(inverted_index):
    """
    Create skip pointers for the inverted index.

    Parameters:
    inverted_index (dict): An inverted index of the documents

    Returns:
    dict: An inverted index of the documents with skip pointers

    """
    
    #  create skip pointers
    skip_inv_index = {}
    for word, doc_list in inverted_index.items():

        #  create a skip list for each posting list
        skip_list = create_skip_lists(doc_list)

        #  update the posting list with the skip list
        skip_inv_index[word] = skip_list

    return skip_inv_index 

In [329]:
def merge_posting_list_with_skip_pointers(query1, query2, inverted_index):

    """ 

    Function to merge the posting lists of two words using skip pointers


        Parameters:
        query1 (str): first word in the query
        query2 (str): second word in the query
        inverted_index (dict): inverted index of the documents

        Returns:
        list: merged posting list
        
    """

    #  find the posting list for the first word in the query
    posting_list1 = inverted_index[query1]

    #  find the posting list for the second word in the query
    posting_list2 = inverted_index[query2]

    #  create a list to store the merged posting list
    merged_posting_list = []

    i = 0
    j = 0

    #  merge the posting lists
    while i < len(posting_list1) and j < len(posting_list2):
 

        # if the document id of the first posting list is equal to the document id of the second posting list, add the document id to the merged posting list
        if posting_list1[i][0] == posting_list2[j][0]:
            merged_posting_list.append(posting_list1[i][0])
            i += 1
            j += 1
        
        
        
        # if the document id of the first posting list is less than the document id of the second posting list, increment the first posting list index
        elif posting_list1[i][0] < posting_list2[j][0]:
            
            # if the skip pointer of the first posting list is not 0 and the document id of the skip pointer is less than the document id of the second posting list, update the first posting list index
            if posting_list1[i][1] != 0 and posting_list1[i][1] < len(posting_list1) and posting_list1[posting_list1[i][1]][0] <= posting_list2[j][0]:
                
                #  update the first posting list index
                while posting_list1[i][1] != 0 and posting_list1[i][1] < len(posting_list1) and posting_list1[posting_list1[i][1]][0] <= posting_list2[j][0]:
                    i = posting_list1[i][1]

            else:
                i += 1
        
        # if the document id of the first posting list is greater than the document id of the second posting list, increment the second posting list index
        else:

            # if the skip pointer of the second posting list is not 0 and the document id of the skip pointer is less than the document id of the first posting list, update the second posting list index
            if posting_list2[j][1] != 0 and posting_list2[j][1] < len(posting_list2) and posting_list2[posting_list2[j][1]][0] <= posting_list1[i][0]:
                
                # update the second posting list index
                while posting_list2[j][1] != 0 and posting_list2[j][1] < len(posting_list2) and posting_list2[posting_list2[j][1]][0] <= posting_list1[i][0]:
                    j = posting_list2[j][1]

            else:
                j += 1
    
    return merged_posting_list


In [330]:
def search_with_skip_pointers(queries, inverted_index):

    """
    Search for documents using the inverted index with skip pointers.

    Parameters:

    queries (list): A list of queries
    inverted_index (dict): An inverted index of the documents

    Returns:
    list: A list of documents that match the queries

    """

    #  check if the queries are in the inverted index, if not return an empty list
    for query in queries:
        if query not in inverted_index:
            return []
    
    # if only one query is given, return the posting list of the query
    if len(queries) == 1:
        return inverted_index[queries[0]]

    result = []

    #  merge the posting lists of the first two queries
    result = merge_posting_list_with_skip_pointers(queries[0], queries[1], inverted_index)

    #  merge the posting lists of the remaining queries
    for i in range(2, len(queries)):
        result = merge_posting_list_with_skip_pointers(result, queries[i], inverted_index)
    
    return result
    

In [331]:
def time_taken_with_skip_pointers(queries, inverted_index):

    """
    Calculate the time taken to search for documents using the inverted index with skip pointers.

    Parameters:

    queries (list): A list of queries
    inverted_index (dict): An inverted index of the documents

    Returns:
    float: The time taken to search for documents using the inverted index with skip pointers

    """

    #  start the timer
    start = time.time()
    res = []
    for i in range(100):
        res = search_with_skip_pointers(queries, inverted_index)
    
    print(res)
    
    #  stop the timer
    end = time.time()

    #  calculate the time taken
    time_taken = end - start

    # avg time taken
    avg_time_taken = time_taken / 100

    return avg_time_taken


In [332]:
#  create skip pointers for the inverted index
inverted_index_with_skip_pointers = create_skip_pointers(inverted_index)

In [71]:
#  save the inverted index with skip pointers in a text file
with open('inverted_index_with_skip_pointers.json', 'w') as file:
    file.write(json.dumps(inverted_index_with_skip_pointers))

In [94]:
# queries_to_search = ["Tiger AND Safari", "Wildlife AND Poaching", "Leopard AND Night"]

In [333]:
queries = []

#  extract the words from the queries
for query in sample_queries:
    queries.append(extract_queries(query))

In [334]:
#  calculate the time taken to search for documents using the inverted index with skip pointers
time_taken = []
for query in queries:
    time_taken.append(time_taken_with_skip_pointers(query, inverted_index_with_skip_pointers))

print("Time taken to search for documents using the inverted index with skip pointers: ", time_taken)

[62, 44, 25, 65, 66, 47, 67, 48, 68, 69, 30, 80]
[]
[71]
Time taken to search for documents using the inverted index with skip pointers:  [8.013248443603516e-05, 0.0, 4.952192306518555e-05]


# Spelling Correction
Create a 3-gram index for the same documents crawled in part 1. Consider the modified
queries with spelling mistakes :

1. Tiger AND Saphari
2. Wyldlife AND Poching
3. Leprd AND Night

Implement the spelling correction technique on query terms using the 3-gram index.

After that run the same algorithm developed in part 2 to extract the relevant documents.

## Function to create the 3-gram index for the documents

In [335]:
#Building up of trigrams from present words
def make_trigrams():

  """ 
  This function will make trigrams from the words present in the inverted index

  Parameters:
  None

  Returns:
  trigrams (dict): A dictionary of trigrams and the words that contain them

  """
  trigrams = {}

  # traverse through the inverted index
  for i in sorted_inverted_index.keys():

    # if the length of the word is less than 3, skip it
    if(len(i)<3):
      continue

    # if the length of the word is greater than 3, make trigrams
    else:
      # traverse through the word
      j=0

      # loop until the last trigram is made
      while((j+2)<len(i)):

        # make the trigram
        tri = i[j] + i[j+1] + i[j+2]

        # if the trigram is already present in the dictionary, append the word to the list of words that contain the trigram
        if tri in trigrams.keys():
          trigrams[tri].append(i)
        # if the trigram is not present in the dictionary, add the trigram to the dictionary
        else:
          trigrams[tri] = [i]

        # increment the index
        j = j+1
  

  print('final', trigrams)
  
  return trigrams


In [336]:
tri_grams = make_trigrams()

final {'ind': ['india', 'indian', 'find', 'findings', 'bigcatsindia', 'indeed', 'vindhya', 'hind', 'behind', 'mind'], 'ndi': ['india', 'indian', 'findings', 'surrounding', 'spending', 'understanding', 'bigcatsindia', 'undisputed', 'expanding', 'conditions', 'ending'], 'dia': ['india', 'indian', 'bigcatsindia'], 'tig': ['tiger', 'tigers'], 'ige': ['tiger', 'tigers'], 'ger': ['tiger', 'tigers', 'endangered', 'passenger', 'loggers', 'badger'], 'saf': ['safari', 'safaris', 'safe', 'bigcatssafari', 'safely', 'safest'], 'afa': ['safari', 'safaris', 'bigcatssafari'], 'far': ['safari', 'safaris', 'welfare', 'bigcatssafari', 'far'], 'ari': ['safari', 'safaris', 'various', 'variety', 'sharing', 'sanctuaries', 'bigcatssafari', 'variables', 'clearias', 'tadobaandhari', 'andhari', 'itineraries'], 'nat': ['national', 'nature', 'natural', 'naturalists', 'international', 'designated', 'karnataka', 'internationally', 'adityanath', 'naturalist', 'nations', 'naturally', 'jonathan', 'examination', 'supern

## Function to merge the posting lists using trigram index

In [337]:
#Merge algo of 2 slightly changed to find intersection of trigram lists
def merge_algo_2(queries):

  """ 
  This function will merge the posting lists of the queries using the merge algorithm 2 ie. using the trigrams

  Parameters:
  queries (list): A list of queries

  Returns:
  list: A list of documents that match the queries

  """

  answer_postings = []
  
  # check if the queries are in the inverted index, if not return an empty list
  for i in queries:
    if tri_grams.get(i) == None:
      return []
    else:
      answer_postings.append(tri_grams[i])

  # help restrictive intersection first
  answer_postings.sort(key=len)

  # sort the posting lists
  for i in answer_postings:
    i.sort()
  
  # get the first posting list
  top = answer_postings[0]

  # traverse through the remaining posting lists
  for i in range(1, len(answer_postings)):
    #intersection with first list
    m = len(top)
    
    n = len(answer_postings[i])
    
    k=0
    l=0
    merged = []

    # merge the posting lists
    while(k<m and l<n):

      # if the posting lists have the same document id, append it to the merged list
      if(top[k] == answer_postings[i][l]):
        merged.append(top[k])
        k = k+1
        l = l+1
      
      else:
        # if the document id of the first posting list is less than the document id of the second posting list, increment the index of the first posting list
        if(top[k] < answer_postings[i][l]):
          k = k+1
        # if the document id of the first posting list is greater than the document id of the second posting list, increment the index of the second posting list
        else:
          l=l+1

    top = merged

  return top

One of the assumptions taken is that the word must contain atleast two correct trigrams, so that intersection of the trigrams of the words exists

In [338]:
def spell_check(wrong_word):

  """ 
  This function will find the correct spelling of the word

  Parameters:
  wrong_word (str): The word whose spelling is to be checked

  Returns:
  list: A list of words that are similar to the word whose spelling is to be checked
  
  """

  j=0
  word_tri = []

  #make trigrams of the word
  while((j+2)<len(wrong_word)):

    # make the trigram
    tri = wrong_word[j] + wrong_word[j+1] + wrong_word[j+2]

    #append to list
    word_tri.append(tri)
    j=j+1

  i=0

  # if only one trigram is present, return an empty list
  if(len(word_tri)<2):
    print("Spellcheck can't take place")
    return []
  

  intersections = []
  #pairs of trigrams taken to find intersection
  for i in range(len(word_tri)):
    for j in range(i+1, len(word_tri)):
      intersections.append(merge_algo_2([word_tri[i], word_tri[j]]))

  return intersections

In [339]:
#to choose word containing most trigram parts
def most_frequent(List):
    return max(set(List), key = List.count)

In [340]:
def process_query_1d(user_query):

  """

  This function will process the query and return the documents that match the query

  Parameters:
  user_query (str): The query to be processed

  Returns:
  list: A list of documents that match the query

  """
  
  user_query = extract_queries(user_query)

  print("query" , user_query)

  # loop through the query
  for i in range(len(user_query)):

    # if the word is not present in the inverted index, find the correct spelling of the word
    if user_query[i] not in sorted_inverted_index.keys():

      # find the correct spelling of the word
      possibilities = spell_check(user_query[i])
      all_pos = []

      # append all the words that are similar to the word whose spelling is to be checked
      for j in possibilities:
        all_pos = all_pos + j

      # if no words are similar to the word whose spelling is to be checked, return an empty list
      if(len(all_pos)<1):
        user_query[i] = user_query[i]

      # if words are similar to the word whose spelling is to be checked, choose the word that contains the most trigrams
      else:
        user_query[i] = most_frequent(all_pos)

  print(user_query)

  # merge the posting lists of the words in the query
  retrieved_docs = merge_algo(user_query)

  
  print('Retrieved Docs :' , retrieved_docs)
  return retrieved_docs

In [341]:
process_query_1d('Tiger AND Saphari')
#Note we see the corrected word is not Safari, since as per our assumption, the word Saphari has only one trigram correct which is not enough

query ['tiger', 'saphari']
['tiger', 'sharing']
happen
[62, 6]
[21, 62, 23, 44, 25, 65, 66, 27, 47, 67, 28, 48, 68, 69, 30, 51, 71, 32, 52, 13, 33, 54, 74, 35, 37, 78, 39, 40, 80]
Retrieved Docs : [62]


[62]

In [342]:
process_query_1d('Wyldlife AND Poching')
#same as first case, better results could be found if bigram was used instead of trigram

query ['wyldlife', 'poching']
['wildlife', 'watching']
happen
[42]
[41, 42, 23, 43, 4, 24, 45, 65, 66, 27, 47, 48, 69, 30, 70, 52, 54, 74, 56, 37, 77, 18]
Retrieved Docs : [42]


[42]

In [343]:
process_query_1d('Lepard AND Night')

query ['lepard', 'night']
['department', 'night']
happen
[21, 42, 47, 48, 49, 30]
[41, 42, 43, 45, 46, 47, 48, 71, 32, 52, 55, 56, 57]
Retrieved Docs : [42, 47, 48]


[42, 47, 48]

# Scoring
Extend your system from part 2 to perform simple TF-IDF scoring of the retrieved results.


In [344]:
def inverse_document_frequency(term, inverted_index):

    """
    Calculate the inverse document frequency of a term.

    Parameters:

    term (str): A term
    inverted_index (dict): An inverted index of the documents

    Returns:
    float: The inverse document frequency of the term

    """

    #  find the number of documents in the corpus
    N = 80

    #  find the number of documents in the corpus that contain the term
    df = len(inverted_index[term])

    #  calculate the inverse document frequency upto 5 decimal places
    idf = round(math.log10(N / df), 10)

    return idf

In [345]:
def term_frequency(term, doc_id):
    """
    Calculate the term frequency of a term in a document.

    Parameters:

    term (str): A term
    document (str): A document

    Returns:
    int: The term frequency of the term in the document

    """

    FILE_NAME = "documents\d" + str((doc_id - 1) % 20 + 1 ) + "_q" + str((doc_id - 1) // 20 + 1) + ".txt"

    #  open the document
    with open(FILE_NAME, 'r') as file:
        document = file.read()
    
    #  split the document into words
    words = document.split()

    #  count the number of times the term appears in the document
    count = words.count(term)

    # upto 5 decimal places
    if count == 0:
        tf = 0
        
    tf = round(0.5 + 0.5 * count / max(words.count(word) for word in words), 5)

    return tf

In [346]:
def tf_idf(term, doc_id, inverted_index):

    """
    Calculate the tf-idf score of a term in a document.

    Parameters:

    term (str): A term
    document (str): A document
    inverted_index (dict): An inverted index of the documents

    Returns:
    float: The tf-idf score of the term in the document

    """

    #  calculate the term frequency
    tf = term_frequency(term, doc_id)

    #  calculate the inverse document frequency
    idf = inverse_document_frequency(term, inverted_index)

    #  calculate the tf-idf score upto 5 decimal places
    tf_idf = round(tf * idf, 5)

    return tf_idf

In [347]:
# perform simple TF-IDF scoring of the retrieved results

def tf_idf_score(query, doc_id, inverted_index):
    
        """
        Calculate the TF-IDF score of a document.
    
        Parameters:
    
        query (list): A list of words in the query
        document (str): The document to calculate the TF-IDF score of
        inverted_index (dict): An inverted index of the documents
    
        Returns:
        float: The TF-IDF score of the document
    
        """
    
        #  create a list to store the TF-IDF scores of the words in the query
        tf_idf_scores = []
    
        #  calculate the TF-IDF score of each word in the query
        for word in query:
            if word not in inverted_index:
                print(f"{word} not in inverted index")
                return 0
            
            tf_idf_scores.append(tf_idf(word, doc_id, inverted_index))
    
        #  return the sum of the TF-IDF scores
        return sum(tf_idf_scores)

In [348]:
#  find the tf idf of term in documents in the result set of the query
import operator


def cal_tf_idf(query, inverted_index, result):
    """
    Calculate the TF-IDF score of the documents in the result set of the query.

    Parameters:

    query (list): A list of words in the query
    inverted_index (dict): An inverted index of the documents
    result (list): A list of documents that match the query

    Returns:
    list: A list of documents that match the query sorted by the TF-IDF score

    """

    #  create a dictionary to store the TF-IDF score of the documents in the result set
    tf_idf_scores = {}

    #  calculate the TF-IDF score of the documents in the result set
    for doc_id in result:
        doc_name = "d" + str((doc_id - 1) % 20 + 1 ) + "_q" + str((doc_id - 1) // 20 + 1)
        tf_idf_scores[doc_name] = tf_idf_score(query, doc_id, inverted_index)

    #  sort the documents in the result set by the TF-IDF score
    sorted_tf_idf_scores = sorted(tf_idf_scores.items(), key=operator.itemgetter(1), reverse=True)

    return sorted_tf_idf_scores

In [351]:
tf_idf_query = []

for i in range(len(queries)):
    print("Query: ", sample_queries[i])

    tf_idf_query.append(cal_tf_idf(queries[i], inverted_index, results[i]))

    print("tf-idf: ", tf_idf_query[i])

Query:  Tiger AND Safari
tf-idf:  [('d20_q4', 0.91242), ('d5_q4', 0.8022499999999999), ('d14_q4', 0.79707), ('d4_q3', 0.7944800000000001), ('d5_q2', 0.7944800000000001), ('d18_q4', 0.76034), ('d2_q4', 0.75518), ('d8_q4', 0.75518), ('d7_q3', 0.72373), ('d10_q2', 0.69621), ('d7_q4', 0.6869000000000001), ('d6_q4', 0.6858599999999999), ('d9_q4', 0.68431), ('d19_q2', 0.62535), ('d8_q3', 0.59617)]
Query:  Wildlife AND Poaching
tf-idf:  []
Query:  Leopard AND Night
tf-idf:  [('d11_q4', 1.0348000000000002)]
